# Interactively Inspecting Studies’ Multi-Modal Sensor Data

---
by [Kristof Van Laerhoven](https://orcid.org/0000-0001-5296-5347), as developed in the [SPP2199](https://scalableinteractionparadigms.uni-oldenburg.de/) Winter School in 2025. DOI: [`10.5281/zenodo.14915796`](https://zenodo.org/records/14915796)





The aim of this notebook is to explain concisely how to set up an interactive timeseries plot to browse large amounts of sensor data. We start by explaining the problem, and then give a minimal example how to set up a plot with just a few lines of python and javascript code.

# 1. Loading sensor data
We start simple with ECG signal data (sample 200 from svdb, provided by [PhysioNet](https://physionet.org/about/database/)).
These data represent a single, normal heartbeat:

In [ ]:
myData = [ 0, 0, 0.01, -0.03, -0.01, -0.04, -0.04, -0.03, -0.05, -0.02,
          -0.04,  0, -0.03, -0.01, -0.04,  0.02, -0.06, -0.04, -0.01,
          -0.03, 0.01,  0.05,  0.05,  0.11, 0.18,  0.16,  0.25,  0.21,
          0.18,  0.09,  0.23,  0.06,  0.15,  0.08, -0.03,  0, -0.02,
          -0.09, -0.08, -0.04, -0.05,  0.01,  0.15,  0.16,  0.63,  0.77,
          1.15,  0.51, -0.13, -0.77, -1.41, -1.25, -0.87, -0.59, -0.32,
          -0.27, -0.24, -0.23, -0.19, -0.2, -0.18, -0.17, -0.17, -0.17,
          -0.16, -0.15, -0.15, -0.15, -0.21, -0.13, -0.17, -0.12, -0.13,
          -0.13, -0.1, -0.06, -0.08, -0.08, -0.04, -0.04,  0, 0, 0.06,
          0.06, 0.12, 0.11, 0.13, 0.21, 0.27, 0.23, 0.28, 0.2, 0.25, 0.15 ]


# 2. Visual inspection
How do we know these data are valid and the recording worked as advertised?

If we know the sensor's behavior, we can plot the resulting *timeseries* and see if it matches our expectation. Plotting it this way shows something you'll probably be somewhat familiar with. There are two common ways:

**1. Statically:** One typical way to quickly plot the data, is by creating a line plot over all samples using [matplotlib](https://matplotlib.org/):


In [ ]:
import matplotlib.pyplot as plt
plt.plot(myData, marker='.', linestyle='solid' )
plt.show()

**2. Interactively**: [plotly](https://plotly.com) allows to move around the plot with the mouse cursor. Click-and-drag selects a region to zoom into, double-click zooms out to the original plot. Let's also connect the data samples with splines for a smoother timeseries:

In [ ]:
import plotly.express
plotly.express.line(myData, line_shape='spline')

# 3. The Problem: Scaled-up Timeseries
Now we increase the size of our data, by repeating this timeseries *n* times, and adding some noise to create a semi-realistic heart beat timeseries:


In [ ]:
import numpy as np
n = 7000
myData = np.tile(myData, n)+0.05*np.random.randn(len(myData)*n)
print(myData)


Now go back to [Section 2](#scrollTo=lz4CjOuYmyOb) above, to plot this new, larger amount of data (Note that we overwrite the myData variable). Is the plotly visualization still as interactive?
⬆



---



The more data, the slower such timeseries plots become. This is aggrevated even more when more sensors produce your (multidimensional) data.
  


# 4. Preparing for uPlot
The surprisingly quick alternative to plotting your timeseries data with methods such as the above is to use a browser-based javascript library.

We'll need to first modify the data into an array of quantized values. For this, calculate the minimum and the scale of your timeseries:


In [ ]:
offset = np.amin(myData)
scale  = np.amax(myData)-offset
print([offset,scale])
print(myData)
myJSData = [int(99*(x-offset)/scale) for x in myData]
myJSData

And then save the timeseries as a .js file:


In [ ]:
with open('data.js',"w") as f:
  f.write("var data=[")
  f.write(",".join( list( map (str, myJSData) ) ) )
  f.write("];\n")

# 5. Plotting with uPlot

Now we create an `index.html` file which, for brevity, includes the javascript code to plot our data with [uPlot](https://github.com/leeoniya/uPlot).

The code below uses a small hack in that it uses an event to run a callback function when the script is loaded. This is guaranteed to run whatever is in the lambda function plotData *after* the data in `data.js` is loaded.


In [ ]:
html = """
<html>
	<head>
		<script src="https://leeoniya.github.io/uPlot/dist/uPlot.iife.min.js"></script>
		<link rel="stylesheet" href="https://leeoniya.github.io/uPlot/dist/uPlot.min.css"/>
	</head>
  <body>
	<script>

    function loadScript(url, callback) {  // load js file with callback:
      var script = document.createElement("script");
      script.src = url;
      script.onreadystatechange = callback;
      script.onload = callback;
      document.head.appendChild(script);
    }

    var plotData = function () {  // callback with everything for plotting:
      const ids = [...Array(data.length).keys()];  // [0,1,2,...,data.length-1]
      new uPlot({  // uPlot options come here:
        id: "chart", width: window.innerWidth, height: 420,
        scales: { auto: false, x: { time: false } },
        series: [ { fill: false },{ label: "ecg", stroke: "red" } ]
        }, [ids, data], document.body);
    }

    loadScript("data.js", plotData);  // first load data.js, then call plotData

  </script></body>
</html>
"""

htmlFile= open("index.html","w")
htmlFile.write(html)
htmlFile.close()

uPlot is a highly efficient interactive JS plotting library that will be able to handle millions of data points.

Download the `data.js` and the `index.html` files and open the latter in your browser to start the inspection:





In [ ]:
from google.colab import files
files.download('data.js')
files.download('index.html')

You can go back and create a larger dataset by changing the value of *n* in Section 3 and see its interactive speed. **Warning:** This notebook might crash if you set the value of *n* too large. A value of 100000 should just about work in this notebook (with lengthy delays).
⬆


---



# 6. Examples

Here are two examples made using the above principles:

*   [Wear Dataset Visualization](https://kristofvl.github.io/wearviz): This website uses an HTML5 video element, which is synchronized in the time axis with a 12-dimensional timeseries. Zooming can be done with the mouse scroll button.
*   [WESAD Dataset Visualization](https://kristofvl.github.io/wesadviz): This website uses two synchronized plots from two different wearable sensing systems.   

Their respective GitHub repositories:
*   [WearViz](https://www.github.com/kristofvl/wearviz)
*   [WESADViz](https://www.github.com/kristofvl/wesadviz)



# 7. Miscellaneous

Interesting further topics from here:
* [Plotly Resampler](https://github.com/predict-idlab/plotly-resampler): visualize large sequential data by adding resampling functionality to Plotly figures in Python
* [STUMPY](https://stumpy.readthedocs.io/en/latest/Tutorial_The_Matrix_Profile.html) a very fast library to obtain a matrix profile, a vector holding the z-normalized Euclidean distance between any subsequence within a time series and its nearest neighbor. Highly recommended for finding certain patterns in time series.
* Using more advanced methods, such as FPCS: H. Li, B. Yang and Y. Chua, "FPCS: Feature Preserving Compensated Sampling of Streaming Time Series Data" doi: [10.1109/TVCG.2024.3456375](https://ieeexplore.ieee.org/document/10669793/)
---


This notebook was produced as part of the PervaSafe project, which is funded by the Deutsche Forschungsgemeinschaft (DFG, German Research Foundation) - 425868829 and is part of Priority Program
SPP2199 Scalable Interaction Paradigms for Pervasive Computing Environments.
